<a href="https://colab.research.google.com/github/yuguerten/Casablanca-housing/blob/main/Real_Estate_Data_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Installing libraries*

In [2]:
!pip install beautifulsoup4
!pip install requests

*Improting libraries*

In [30]:
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt


*Define the url and use the GET html method to extract the web page*

In [62]:
url = "https://www.mubawab.ma/fr/scr/casablanca-settat/appartements-a-vendre:o:d"
res = requests.get(url)

In [63]:
print(res)

<Response [200]>


*Using BeautifulSoup to parse the response variable*

In [ ]:
soup = BeautifulSoup(res.text)
soup

*Use BeautifulSoup's select function to scrape the desired content*

In [ ]:
housing = soup.select(".listingH3 , #mainListing .floatR , .listingTit a , .hardShadow")
housing

*Scrapping the data of the 1st page*

In [66]:
# Initialize lists to store data
location = []
total_rooms= []
description = []
prices = []

for i in range(0, len(housing), 5):
    location.append(housing[i+4].get_text(strip=True))
    total_rooms.append(housing[i + 3].get_text(strip=True))
    description.append(housing[i + 2].get_text(strip=True))
    prices.append(housing[i + 1].get_text(strip=True))

for i in range(len(location)):
    print(f"Location: {location[i]}")
    print(f"Total Rooms: {total_rooms[i]}")
    print(f"Description: {description[i]}")
    print(f"Price: {prices[i]}")
    print("\n")

Location: Oulfa à
   					Casablanca
Total Rooms: 3 chambres, 116 m²
Description: Duplex deux facades sud/est bien ensoleil...
Price: 980 000 000 DH


Location: Casablanca Marina à
   					Casablanca
Total Rooms: 2 chambres, 142 m²
Description: Appartement marina vue mer mosquée hassan...
Price: 530 000 000 DH


Location: Ain Diab à
   					Casablanca
Total Rooms: 3 chambres, 280 m²
Description: Appartement à vendre à Ain Diab. 5 grande...
Price: 450 000 000 DH


Location: Anfa Supérieur à
   					Casablanca
Total Rooms: 3 chambres, 110 m²
Description: Opportunité à saisir
Price: 165 000 000 DH


Location: Les Hôpitaux à
   					Casablanca
Total Rooms: 2 chambres, 113 m²
Description: Appartement à vendre à Les Hôpitaux. Surf...
Price: 156 000 000 DH


Location: Les Hôpitaux à
   					Casablanca
Total Rooms: 2 chambres, 109 m²
Description: Vente d'un bel appartement à Les Hôpitaux...
Price: 140 000 000 DH


Location: Les princesses à
   					Casablanca
Total Rooms: 1 chambre, 55 m²
Desc

In [67]:
# for price in prices:
#   print(price)
len(prices)

33

*scrape the data of all the pages*

In [97]:
base_url = "https://www.mubawab.ma/fr/scr/casablanca-settat/appartements-a-vendre:o:d:p"
housing = soup.select(".listingH3 , #mainListing .floatR , .listingTit a , .hardShadow")
number_of_pages = 18
for i in range(2,number_of_pages+1):
    url = f"{base_url}:{i}" # URL Manupilation for each page
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    for j in range(0, len(housing), 5):
      prices.append(housing[j + 1].get_text(strip=True))
      description.append(housing[j + 2].get_text(strip=True))
      total_rooms.append(housing[j + 3].get_text(strip=True))
      location.append(housing[j+4].get_text(strip=True))

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [98]:
# for i in range(len(location)):
#     print(f"Location: {location[i]}")
#     print(f"Total Rooms: {total_rooms[i]}")
#     print(f"Description: {description[i]}")
#     print(f"Price: {prices[i]}")
#     print("\n")

len(prices)

1155

In [ ]:
# Function to convert a price string to an integer
def convert_price(price_str):
    try:
        # Remove non-breaking space characters and other non-numeric characters
        cleaned_price_str = ''.join(filter(str.isdigit, price_str))
        return int(cleaned_price_str)
    except ValueError:
        return float('inf')  # Handle cases where the price is not a valid number

# Convert prices to a list of tuples (original index, price) for sorting
prices_with_indices = list(enumerate(prices))

# Sort the list of tuples based on the price
sorted_prices_with_indices = sorted(prices_with_indices, key=lambda x: convert_price(x[1]))

# Display the sorted prices
for index, price in sorted_prices_with_indices:
    print(f"{index + 1}. {price}")

In [ ]:
for total_room in total_rooms:
  print(total_room)

In [111]:
def extract_bedrooms_area(info_str):
    # Use regular expression to extract the number of bedrooms and area
    match = re.search(r'(\d+)?\s*chambres,\s*(\d+)?\s*m²', info_str)

    # Extract bedrooms and area or return None for missing values
    bedrooms = match.group(1) if match and match.group(1) else None
    area = match.group(2) if match and match.group(2) else None

    return bedrooms, area
bedrooms_list, area_list = zip(*[extract_bedrooms_area(info) for info in total_rooms])


In [ ]:
area_list

In [102]:
import re
# Function to extract numerical values from the price string
def extract_numerical_value(price_str):
    # Use regular expression to extract digits and convert to an integer
    match = re.search(r'\d+', price_str)
    return int(match.group()) if match else None

# Extract numerical values from prices
numeric_prices = [extract_numerical_value(price) for price in prices]

In [114]:
# Function to extract numeric values from the num_rooms string
def extract_numeric_rooms(num_rooms_str):
    if isinstance(num_rooms_str, str):
        # Use regular expression to extract digits and convert to an integer
        match = re.search(r'\d+', num_rooms_str)
        return int(match.group()) if match else None
    else:
        return None


# Extract numeric values from num_rooms
numeric_rooms = [extract_numeric_rooms(room) for room in bedrooms_list]

In [115]:
# Function to extract numeric values from the area string
def extract_numeric_area(area_str):
    if isinstance(area_str, str):
        # Use regular expression to extract digits and convert to an integer
        match = re.search(r'\d+', area_str)
        return int(match.group()) if match else None
    else:
        return None

# Extract numeric values from area
numeric_area = [extract_numeric_area(a) for a in area_list]

In [116]:
print(numeric_area[98])
print(len(location))
print(len(numeric_rooms))
print(len(numeric_area))
print(len(numeric_prices))

509
1155
1155
1155
1155


In [118]:
import pandas as pd
data = {
    'Location' : location,
    'Num_rooms' : numeric_rooms,
    'House_area(m²)' : numeric_area,
    'Price(DH)' : numeric_prices,
}
housing = pd.DataFrame(data)

In [119]:
sorted_housing = housing.sort_values(by='Price(DH)', ascending = False)
sorted_housing

,Location,Num_rooms,House_area(m²),Price(DH)
0,Oulfa à\n \t\t\t\t\tCasablanca,3.0,116.0,980
396,Oulfa à\n \t\t\t\t\tCasablanca,3.0,116.0,980
528,Oulfa à\n \t\t\t\t\tCasablanca,3.0,116.0,980
66,Oulfa à\n \t\t\t\t\tCasablanca,3.0,116.0,980
495,Oulfa à\n \t\t\t\t\tCasablanca,3.0,116.0,980
...,...,...,...,...
774,Maârif à\n \t\t\t\t\tCasablanca,3.0,173.0,3
773,Anfa à\n \t\t\t\t\tCasablanca,2.0,126.0,3
772,à\n \t\t\t\t\tDar Bouazza,3.0,318.0,3
771,à\n \t\t\t\t\tDar Bouazza,3.0,358.0,3


In [120]:
housing[housing['Location'] == "."]

,Location,Num_rooms,House_area(m²),Price(DH)


In [121]:
housing['Location']  = housing['Location'].str.split('\n', expand=True)[0].str.strip()
housing['Location']  = housing['Location'].str.replace('à','\n').str.strip()

housing.head()

,Location,Num_rooms,House_area(m²),Price(DH)
0,Oulfa,3.0,116.0,980
1,Casablanca Marina,2.0,142.0,530
2,Ain Diab,3.0,280.0,450
3,Anfa Supérieur,3.0,110.0,165
4,Les Hôpitaux,2.0,113.0,156


In [122]:
housing.shape

(1155, 4)

In [123]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1155 entries, 0 to 1154
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Location        1155 non-null   object 
 1   Num_rooms       1068 non-null   float64
 2   House_area(m²)  1068 non-null   float64
 3   Price(DH)       1155 non-null   int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 36.2+ KB


In [125]:
casablanca_housing = housing.to_csv('housing_data.csv', index=False)